# Preparando ambiente

## Instalando libs

In [1]:
!pip install --upgrade google-cloud-bigquery -q
!pip install --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]' -q

In [2]:
# bibliotecas
import json
from requests import request
from datetime import datetime
from google.cloud import bigquery
from pandas_gbq import read_gbq, to_gbq
from google.oauth2 import service_account
from pandas import read_excel, read_csv, concat, merge, DataFrame, to_datetime, read_gbq,read_sql_query

# KarHub Alias

## Consumo dos dados

In [3]:
produtos_alias = read_csv("Karhub-alias.csv", sep=",")

## Tratamento dos dados

Esta tabela possui um mapeamento entre o código original do produto e o códigos sinônimos. Algumas de suas colunas possuem caracteres especiais como acento e/ou underscore e estão com tipagem incorreta. Antes de realizar as análises e responder as perguntas de negócio é necessário sanitizar este dataset.

In [4]:
produtos_alias.rename(columns={"Código Fabricante": "CodigoFabricante"},inplace=True)
produtos_alias.rename(columns={"Alias": "Codigo"},inplace=True)

columns = list(produtos_alias.columns)

for column in columns:
    #mudando tipagem
    produtos_alias[f"{column}"] =  produtos_alias[f"{column}"].astype("string")
    #renomeando coluna

print(produtos_alias.shape)
print(produtos_alias.dtypes)
produtos_alias.head()

(1250, 3)
Fabricante          string
CodigoFabricante    string
Codigo              string
dtype: object


,Fabricante,CodigoFabricante,Codigo
0,KarHub,FO0352,fo0352
1,KarHub,FO0353,fo0353
2,KarHub,FT0417,ft0417
3,KarHub,VW0127,vw0127
4,KarHub,PE0452,pe0452


# Produtos KarHub

## Consumo dos dados

In [5]:
# produtos_xlsx.to_csv ('produtos', index = None, header=True, sep = ";")

In [6]:
# produtos_part2 = read_csv("karhub_autoparts_1/Tabela2.csv", sep=";")
# produtos_part1 = read_csv("karhub_autoparts_1/Tabela1.csv", sep=";")
produtos = read_excel("karhub_autoparts_1.xlsx")

## Tratamento dos dados

Os dados sobre as peças automotivas/produtos estão em uma planilha de excel. Assim como a tabela com os alias, este conjunto de dados precisa passar um processo de sanitização e limpeza.

In [7]:
#sanitizacao
produtos.columns = [i.replace(" ","").replace("(","").replace(")","") for i in produtos.columns]

#Filtrando registros cujo valos da Coluna NomeAtributo seja Largura (cm" ou Altura (cm)
produtos = produtos[((produtos["NomeAtributo"] == "Largura (cm)") | (produtos["NomeAtributo"] == "Altura (cm)"))]

#renomeando colunas
produtos.rename(columns={"Composição": "Composicao"},inplace=True)
produtos.rename(columns={"Código": "Codigo"},inplace=True)

for i,column in enumerate(produtos.columns):
    if column == f"Unnamed:{i}":
        del produtos[f"{column}"]
    elif column == "ValorAtributo":
        produtos[f"{column}"] = [i.replace(",",".") for i in produtos[f"{column}"]]
        produtos[f"{column}"] = produtos[f"{column}"].astype("float")
    else:
        produtos[f"{column}"] =  produtos[f"{column}"].astype("string")

#registros com valores de altura
df_alt = produtos.filter(items=['NomeAtributo', 'Codigo'])
df_alt["Altura_cm"] = produtos[produtos["NomeAtributo"] == "Altura (cm)"]["ValorAtributo"]
df_alt.dropna(inplace=True)

#registros com valores de largura
df_larg = produtos.filter(items=['NomeAtributo', 'Codigo'])
df_larg["Largura_cm"] = produtos[produtos["NomeAtributo"] == "Largura (cm)"]["ValorAtributo"]
df_larg.dropna(inplace=True)

#drop de colunas
df_alt.drop("NomeAtributo", inplace = True, axis = 1)
df_larg.drop("NomeAtributo", inplace = True, axis = 1)

produtos.drop("ValorAtributo", inplace = True, axis = 1)
produtos.drop("NomeAtributo", inplace = True, axis = 1)

#merge dos registros com valores de largura e altura
df = df_larg.merge(df_alt, on="Codigo")

#merge final
produtos = produtos.merge(df, on="Codigo")

print(produtos.shape)
print(produtos.dtypes)
produtos.head()

(1006, 7)
NomeSKU        string
Fabricante     string
Codigo         string
Composicao     string
Categoria      string
Largura_cm    float64
Altura_cm     float64
dtype: object


,NomeSKU,Fabricante,Codigo,Composicao,Categoria,Largura_cm,Altura_cm
0,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,18.0,9.0
1,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,18.0,9.0
2,Mola da Suspensão Dianteira,KarHub,fo0353,UNITARY,Mola Helicoidal,18.0,9.0
3,Mola da Suspensão Dianteira,KarHub,fo0353,UNITARY,Mola Helicoidal,18.0,9.0
4,Mola da Suspensão Traseira,KarHub,ft0417,UNITARY,Mola Helicoidal,18.0,9.0


# API de veículos compatíveis

## Autenticação

A API que disponibiliza os dados dos automóveis não é pública, e portanto precisa de autenticação para obter os dados na requisição GET. Para obter o token de acesso fiz uma requisição POST utilizando a biblioteca requests no endpoint disponibilizado, com os parametros pedidos no corpo da requisição. O token é o valor da chave API TOKEN, do json que vem no corpo do response da API.

In [8]:
url_auth = "https://api-data-engineer-test-3bqvkbbykq-uc.a.run.app/user/"

payload = json.dumps({
  "full_name": "Tatiane Correa da Costa e Silva",
  "email": "correa.tcs@gmail.com"
})
headers = {
  'Content-Type': 'application/json'
}

# response = request("POST", url_auth, headers=headers, data=payload)

response = {
    "user": "correa.tcs@gmail.com",
    "API Token": "KH8ZCB4IO79X"
}

token = response["API Token"]

## Consumo dos dados

In [9]:
url = f"https://api-data-engineer-test-3bqvkbbykq-uc.a.run.app/token={token}"

payload = ""
headers = {}

response = request("GET", url, headers=headers, data=payload)
data = response.json()

## Tratamento dos dados

Os dados obtidos via requisição na API estão no formato json, e através de uma iteração foi possível transformar em um dataframe. Este conjunto de dados, assim como os outros, precisa de uma sanitização para poder ser consumido.

In [10]:
fabricante = []
codigo = []
marca = []
modelo = []
ano = []
complemento = []

#iteracao na lista de jsons para criar o dataframe
data  = response.json()
for content in data:
    fabricante.append(content["Fabricante"])
    codigo.append(content["Código Fabricante"])
    marca.append(content["Marca"])
    modelo.append(content["Modelo"])
    ano.append(content["Ano"])
    complemento.append(content["Complemento"])
    
values = list(zip(fabricante,codigo,marca,modelo,ano,complemento))

#nomeando as colunas
veiculos_compativeis = DataFrame(values, columns=['Fabricante', 'Código Fabricante','Marca','Modelo','Ano','Complemento'])

#tipagem
veiculos_compativeis.Ano = veiculos_compativeis.Ano.astype("string")

veiculos_compativeis.dropna(inplace=True)

#sanitização
veiculos_compativeis.Ano = [i.replace(".0","") for i in veiculos_compativeis.Ano]
veiculos_compativeis.Complemento = [i.split("  | ") for i in veiculos_compativeis.Complemento]

#gerando lista de colunas
columns = list(veiculos_compativeis.columns)

#explodindo coluna complemento
veiculos_compativeis = veiculos_compativeis.explode("Complemento")

for i,column in enumerate(veiculos_compativeis.columns):
    if column == "Código Fabricante":
        veiculos_compativeis.rename(columns={"Código Fabricante": "CodigoFabricante"},inplace=True)
        veiculos_compativeis["CodigoFabricante"] =  veiculos_compativeis["CodigoFabricante"].astype("string")        
    elif column == "Ano":
        veiculos_compativeis[f"{column}"] =  to_datetime(veiculos_compativeis[f"{column}"]).dt.year
    else:
        veiculos_compativeis[f"{column}"] =  veiculos_compativeis[f"{column}"].astype("string")

print(veiculos_compativeis.shape)
print(veiculos_compativeis.dtypes)
veiculos_compativeis.head()

(27477, 6)
Fabricante          string
CodigoFabricante    string
Marca               string
Modelo              string
Ano                  int64
Complemento         string
dtype: object


,Fabricante,CodigoFabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 2P
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 4P
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 2P
0,KarHub,FO0352,FORD,FIESTA,1996,1.3 MPI 8V CLX 4P
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 4P


# Produtos x Veículos compatíveis

## Tratamento dos dados

In [11]:
#merges

produtos_alias_merged = produtos.merge(produtos_alias, on="Codigo",suffixes=["","-"])
produtos_alias_merged.drop(['Fabricante-'], axis=1,inplace=True)

produtos_alias_merged
produtos_merged = veiculos_compativeis.merge(produtos_alias_merged, on="CodigoFabricante",suffixes=["","-"])
produtos_merged.drop(['Fabricante-'], axis=1,inplace=True)

produtos_merged = produtos_merged.assign(candidate_name='Tatiane Corrêa da Costa e Silva')

produtos_merged.candidate_name = produtos_merged.candidate_name.astype("string")

produtos_merged = produtos_merged.assign(df_insert=datetime.now())

print(produtos_merged.shape)
print(produtos_merged.dtypes)
produtos_merged.head()

(56722, 14)
Fabricante                  string
CodigoFabricante            string
Marca                       string
Modelo                      string
Ano                          int64
Complemento                 string
NomeSKU                     string
Codigo                      string
Composicao                  string
Categoria                   string
Largura_cm                 float64
Altura_cm                  float64
candidate_name              string
df_insert           datetime64[ns]
dtype: object


,Fabricante,CodigoFabricante,Marca,Modelo,Ano,Complemento,NomeSKU,Codigo,Composicao,Categoria,Largura_cm,Altura_cm,candidate_name,df_insert
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,fo0352,UNITARY,Mola Helicoidal,18.0,9.0,Tatiane Corrêa da Costa e Silva,2023-04-28 20:12:11.513695
1,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,fo0352,UNITARY,Mola Helicoidal,18.0,9.0,Tatiane Corrêa da Costa e Silva,2023-04-28 20:12:11.513695
2,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 4P,Mola da Suspensão Traseira,fo0352,UNITARY,Mola Helicoidal,18.0,9.0,Tatiane Corrêa da Costa e Silva,2023-04-28 20:12:11.513695
3,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 4P,Mola da Suspensão Traseira,fo0352,UNITARY,Mola Helicoidal,18.0,9.0,Tatiane Corrêa da Costa e Silva,2023-04-28 20:12:11.513695
4,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 2P,Mola da Suspensão Traseira,fo0352,UNITARY,Mola Helicoidal,18.0,9.0,Tatiane Corrêa da Costa e Silva,2023-04-28 20:12:11.513695


## Ingestão no BigQuery

Para ingerir dados no BQ é precisso ter algumas permissões do IAM de uso e escrita de tabelas, assim como criação de jobs etc. O método de autenticação utilizado foi o OAuth2 via uma Service Account disponibilizada previamente, com as devidas roles do IAM permitidas.

In [12]:
project_id = "karhub-data-engineer-test"
dataset_id = "cadastro_produto"
table_name = "kh_data_engineer_teste_Tatiane"
table_id = f"{dataset_id}.{table_name}"

In [13]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/tcsilva/workspace/pessoal/estudos/desafio-de-karhub/svc-karhub-data-engineer-test.json',
)

In [14]:
to_gbq(produtos_merged, table_id, project_id=project_id,if_exists='replace', credentials=credentials)

100%|███████████████████████████████████████████████| 1/1 [00:00<00:00, 3086.32it/s]


## Questionário

### 1. </b> Quantos produtos únicos temos na base?<br>

Para responder essa pergunta é necessário contar a quantidade de registros distintos na coluna CodigoFabricante, que é o código original dos produtos/preças automotivas.

In [15]:
sql = f'''
    SELECT 
        COUNT(DISTINCT(CodigoFabricante)) AS total_produtos_unicos
    FROM {dataset_id}.{table_name}
        
        '''

df = read_gbq(sql, project_id=project_id, credentials=credentials)
df

,total_produtos_unicos
0,349


### 2. Quantos veículos únicos (mesma marca e modelo) temos na base por produto?</b><br>
Para responder essa pergunta é necessário contar a quantidade de registros distintos em relação as colunas Marca e Modelo dos automóveis e agrupar pela coluna CodigoFabricante, que é o código original dos produtos/preças automotivas, que é o código original dos produtos/preças automotivas

In [16]:
sql = f"""
    SELECT 
        CodigoFabricante,
        COUNT(DISTINCT CONCAT(Marca, Modelo)) AS n_veiculos_unicos
    FROM {dataset_id}.{table_name}
    GROUP BY CodigoFabricante;"""


df = read_gbq(sql, project_id=project_id, credentials=credentials)
df

,CodigoFabricante,n_veiculos_unicos
0,I-CH0500,1
1,I-FT0011,1
2,I-FT0375,2
3,I-FT0487,1
4,I-HO0597,1
...,...,...
344,I-VW2036,1
345,I-VW2037,1
346,I-VW2042,2
347,I-VW2043,2


### 3. Quantos produtos únicos temos na base por categoria?</b><br>
Para responder essa pergunta é necessário contar a quantidade de registros distintos em relação a coluna CodigoFabricante, que é o código original dos produtos/preças automotivas, e agrupar pela coluna Categoria.

In [17]:
sql = f'''
    SELECT 
        Categoria,
        COUNT(DISTINCT(CodigoFabricante)) AS total_produtos_unicos
    FROM {dataset_id}.{table_name}
    GROUP BY Categoria
'''

df = read_gbq(sql, project_id=project_id, credentials=credentials)
df

,Categoria,total_produtos_unicos
0,Transmissão,17
1,Sonda Lambda,14
2,Freio,11
3,Mola Helicoidal,307
